# Natural Language Processing with RNNs and Attention

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

2024-03-11 00:09:24.692676: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 00:09:24.734324: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 00:09:24.734860: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 00:09:25.632369: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Generating Shakespearean Text Using a Character RNN

In 2015 blog post, Andrej Karpathy showed how RNN can be used to train a model to predict the next character in the sequence. We will look at how to build Char-RNN, step by step, starting with creation of dataset.

### Creating the Training Dataset

First let's download all of Shakespere's work, using Keras's handy `get_file()` function and downloading the data from Andrej Karpathy's Char-RNN Project:

In [2]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespere.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [4]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [5]:
len("".join(sorted(set(shakespeare_text.lower()))))

39

Next, we must encode every character as an integer. One option is to create a custom preprocessing layer as we did in Chapter-13. But in this case, it will be simpler to use Keras's `Tokenizer` class. 

First we need to fit a tokenizer to the text: it will find all the characters used in the text and map each of them to a different character ID, from the 1 to the number of distinct characters (it does not start at 0, as we can use that value for masking, as we will see later):

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) # char_level=True means every character will be treated as a token
tokenizer.fit_on_texts([shakespeare_text])

We set `char_level=True` to get the character-level encoding rather than the default word-level encoding. Note that tokenizer converts the text to lowercase by default (but we can set `lower=False` if we do not want that). Now the tokenizer can encode a sentence (or a list of sentences) to a list of character IDs and back, and it tells how many distinct characters are there and total number of characters in the text:

In [7]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [8]:
tokenizer.sequences_to_texts([[20,6,9,8,3]])

['f i r s t']

In [9]:
max_id = len(tokenizer.word_index) # number of distinct characters
max_id

39

In [10]:
dataset_size = tokenizer.document_count # total number of characters
dataset_size

1

Let's encode the full text so each character is represented by its ID (we subtract 1 to get IDs from 0 to 38, rather than 1 to 39):

In [11]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

Before we continue, we need to split the dataset into training, a validation and a test set. We can't just shuffle all the characters in the text.

### How to split a Sequential Dataset

> Refer notes

Let's take the first 90% of the text for the training set (keeping the rest for the validation set and the test set), and create a `tf.data.Dataset` that will return each character one by one from this set:

In [12]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### Chopping the Sequential Dataset into Multiple Windows

The training set now consists of a single sequence of over a million characters, so we can't just train the neural network directly on it: the RNN would be equivalent to a deep net with over a million layers, and we would have a single (very long) instance to train it. Instead, we will use dataset's `window()` method to convert this long sequence of characters into many smaller windows of text. Every instance in the dataset will be a fairly short substring of the whole text, and the RNN will be unrolled only over the length of these substrings. This is called *truncated backpropogation through time*. 

Let's call the `window()` method to create a dataset of short text windows:

In [13]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

**TIP:**

We can try tuning `n_steps`: it is easier to train RNNs on shorter input sequences, but of course the RNN will not be able to learn any pattern longer than `n_steps`, so don't make it too small.

By default `window()` method creates a nonoverlapping windows, but to get the largest possible training set we use `shift=1` so that the first window contains characters 0 to 100, the second contains 1 to 101, and so on. To ensure that all the windows are exactly 101 characters long (which will allow to create batches without having to do padding), we set `drop_remainder=True` (otherwise the last 100 windows will contain 100 characters, 99 characters, and so on down to 1 character).

The `window()` creates a dataset that contains windows, each of which is also represented as dataset. It is *nested dataset*, analogous to a list of lists. This is useful when we want to transform each window by calling its dataset methods (e.g., to shuffle them or batch them). However, we cannot use a nested dataset directly for training, as our models will expect tensors as input, not datasets. So we must call the `flat_map()` method: it converts nested dataset into *flat dataset* (one that does not contain datasets). Moreover, the `flat_map()` method takes a function as an argument, which allows us to transform each dataset in the nested dataset before flattening. 

In [14]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

Notice that we call the `batch(window_length)` on each window: since all windows have exactly the same length, we will get a single tensor for each of them. Now the dataset contains consecutive windows of 101 characters each. Since GD works best when the instances in the training set are independet and indentically distributed , we need to shuffle these windows. Then we can batch the windows and seperate the inputs (the first 100 characters) from the target (the last character):

In [15]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[: ,1:]))

As discussed in Chapter-13, categorical input features should generally be encoded, usually as one-hot vectors or as embeddings. Here we will encode each character using a one-hot vector because they are fairly few distinct characters (only 39):

In [16]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)

dataset = dataset.prefetch(1)

That's it! Preparing the dataset was the hardest part. Now let's create the model

### Building and Training the Char-RNN Model